In [1]:
'''Interactive run file for the G2-structure learning'''
#...ensure this notebook is using the correct kernel for your virtual environment
# Import libraries
import os
import yaml
import numpy as np
import tensorflow as tf

# Import functions
from models.model import GlobalModel, PatchSubModel
from sampling.sampling import LinkSample
from geometry.compression import form_to_vec, vec_to_form, vec_to_metric
from geometry.patches import PatchChange_Coords, PatchChange_G2form, PatchChange_G2metric
from geometry.geometry import exterior_derivative
from geometry.wedge_product import wedge_product

# Print the hyperparameters
with open(os.getcwd()+'/hyperparameters/hps.yaml', "r") as file:
    hp = yaml.safe_load(file)
print(f'Run hyperparameters:\t...edit in hyperparameters/hps.yaml\n{hp}')

Run hyperparameters:	...edit in hyperparameters/hps.yaml
{'metric': False, 'n_patches': 1, 'overlap_upperwidth': 0.1, 'num_samples': 200000, 'saved_model': False, 'saved_model_path': '...', 'n_hidden': 64, 'n_layers': 3, 'activations': 'gelu', 'use_bias': True, 'epochs': 200, 'batch_size': 100, 'init_learning_rate': 0.005, 'min_learning_rate': 0.005, 'validate': False, 'val_print': False, 'num_val_samples': 500, 'val_batch_size': 100, 'verbosity': 1, 'print_losses': False, 'print_interval': 1}


In [ ]:
# Run the script
model_name = 'test'
!python3 -m run {model_name}

In [2]:
# Import the trained model
#loaded_model_name = model_name #...or set to desired name if not model trained above
#loaded_model_path = os.getcwd() + f"/runs/link_model_{loaded_model_name}.keras" #...reimport the model
loaded_model_path = os.getcwd() + f"/models/link_models/link_model_3form.keras" #...import a pre-trained model instead

# Load the model
loaded_model = tf.keras.models.load_model(loaded_model_path, custom_objects={'GlobalModel': GlobalModel, 'PatchSubModel': PatchSubModel})


absl:WARNING:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


In [3]:
# Test loaded model on fake link points
# Testing hyperparameters
testsize = 100 #...how many link points to use in the testing
metric = False #...whether to learn the G2 3-form (False), or the G2 metric (True)

# Generate the testing data on the Link
test_linkpts, test_link_outputs = LinkSample(testsize, metric=metric)
test_linkpts_tf = tf.convert_to_tensor(test_linkpts)
test_link_outputs_tf = tf.convert_to_tensor(test_link_outputs)
if hp["n_patches"] > 1:
    # Coordinates
    test_linkpts_tf_otherpatches += [PatchChange_Coords(
        test_linkpts_tf[0], 
        output_patch=o_patch,
        ) for o_patch in range(1,5)
        ]
    test_linkpts_tf = tf.stack([test_linkpts_tf] + test_linkpts_tf_otherpatches, axis=0)
    # 3-form
    if not metric:
        test_link_outputs_tf_otherpatches += [PatchChange_G2form(
            test_linkpts_tf[0], 
            test_link_outputs_tf[0],
            output_patch=o_patch,
            ) for o_patch in range(1,5)
            ]
        test_link_outputs_tf = tf.stack([test_link_outputs_tf] + test_link_outputs_tf_otherpatches, axis=0)
    # Metric
    else:
        test_link_outputs_tf += [PatchChange_G2metric(
            test_linkpts_tf[0], 
            test_link_outputs_tf[0],
            output_patch=o_patch,
            ) for o_patch in range(1,5)
            ]
        test_link_outputs_tf = tf.stack([test_link_outputs_tf] + test_link_outputs_tf_otherpatches, axis=0)
else:
    test_linkpts_tf = tf.expand_dims(test_linkpts_tf, axis=0)
    test_link_outputs_tf = tf.expand_dims(test_link_outputs_tf, axis=0)
    
# Compute the NN test outputs      
if not hp["metric"]:
    # Compute the predicted G2 3-forms
    predicted_g2form_vecs = np.array([loaded_model.patch_submodels[patch_idx].predict(test_linkpts_tf[patch_idx]) for patch_idx in range(loaded_model.n_patches)])
    predicted_g2forms = np.array([vec_to_form(predicted_g2form_vecs[patch_idx], 7, 3).numpy() for patch_idx in range(loaded_model.n_patches)])
    print(f'G2 3-forms computed... (shape: {predicted_g2forms.shape})')
else:
    # Compute the predicted G2 metrics
    predicted_g2metric_vecs = np.array([loaded_model.patch_submodels[patch_idx].predict(test_linkpts_tf[patch_idx]) for patch_idx in range(loaded_model.n_patches)])
    predicted_g2metrics = np.array([vec_to_metric(predicted_g2form_vecs[patch_idx], 7, 3).numpy() for patch_idx in range(loaded_model.n_patches)])
    print(f'G2 metrics computed... (shape: {predicted_g2metrics.shape})')

# Compute the MSE loss between computed and NN-predicted values
if not hp["metric"]:
    full_mse_loss = np.mean((test_link_outputs_tf.numpy() - predicted_g2forms) ** 2)
    test_outputs_vecs = np.array([form_to_vec(tsm) for tsm in test_link_outputs_tf])
    vec_mse_loss = np.mean((test_outputs_vecs - predicted_g2form_vecs) ** 2)
else:
    full_mse_loss = np.mean((test_link_outputs_tf.numpy() - predicted_g2metrics) ** 2)
    test_outputs_vecs = np.array([form_to_vec(tsm) for tsm in test_link_outputs_tf])
    vec_mse_loss = np.mean((test_outputs_vecs - predicted_g2metric_vecs) ** 2)
print(f'Test losses (full-form, dof-vector): {(full_mse_loss, vec_mse_loss)}')


/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 4.999999999999999, Vol_cy: 277.6133527745711.


4/4 [==============================] - 0s 645us/step
G2 3-forms computed... (shape: (1, 100, 7, 7, 7))
Test losses (full-form, dof-vector): (0.011376096209209919, 0.01858698582547418)


In [4]:
# Compute exterior derivative of 3-forms over the test data
# Generate test data on the Link to evaluate the identity
identity_test_size = 10
identity_test_linkpts, identity_test_link_outputs, kahler_form = LinkSample(identity_test_size, metric=False, return_kahler_form=True)
identity_test_linkpts_tf = tf.convert_to_tensor(identity_test_linkpts)
if hp["n_patches"] > 1:
    # Coordinates
    identity_test_linkpts_tf_otherpatches += [PatchChange_Coords(
        identity_test_linkpts_tf[0], 
        output_patch=o_patch,
        ) for o_patch in range(1,5)
        ]
    identity_test_linkpts_tf = tf.stack([identity_test_linkpts_tf] + identity_test_linkpts_tf_otherpatches, axis=0)
else:
    identity_test_linkpts_tf = tf.expand_dims(identity_test_linkpts_tf ,axis=0)

# Compute d\phi
dg2_3form = np.array([exterior_derivative(loaded_model.patch_submodels[patch_idx], identity_test_linkpts_tf[patch_idx]) for patch_idx in range(loaded_model.n_patches)])
print(f'G2 3-form exterior derivatives computed... (shape: {dg2_3form.shape})')
###print(f'Non-zero elements: {np.sum(np.where(np.absolute(np.mean(dg2_3form[0],axis=0)) < 1e-5, 1, 0))} / {np.prod(dg2_3form.shape[2:])}')

# Compute omega ^ omega
omega_wedge_omega = np.array([wedge_product(kahler_form[idx], kahler_form[idx]) for idx in range(kahler_form.shape[0])])
print(f'\omega ^ \omega computed... (shape: {omega_wedge_omega.shape})')
###is the vol form specific to a patch? does it change between patches?

# Check whether dg2_3form == omega_wedge_omega
print(f'Checking identity d\phi = \omega ^ \omega:\t{np.all(dg2_3form == omega_wedge_omega)}')


pointgen:INFO:Vol_k: 5.0, Vol_cy: 243.86573769598604.


(10, 7)
(1, 10, 7)
G2 3-form exterior derivatives computed... (shape: (1, 10, 7, 7, 7, 7))
(10, 7, 7)
\Omega ^ \Omega computed... (shape: (10, 7, 7, 7, 7))
Checking identity d\phi = \Omega ^ \Omega:
False


In [14]:
###manual checking -- deleteee
#np.sum(np.where(np.absolute(omega_wedge_omega) > 1e-8, 1, 0)),np.sum(np.where(np.absolute(dg2_3form) > 1e-8, 1, 0))
#print(np.min(dg2_3form), np.mean(dg2_3form), np.max(dg2_3form))
#print(np.min(omega_wedge_omega), np.mean(omega_wedge_omega), np.max(omega_wedge_omega))

for pt_idx in range(identity_test_size):
    print(np.mean(np.absolute(dg2_3form[0][pt_idx] - omega_wedge_omega[pt_idx])))
    ###reduce to non-zero?
    #--> consistently baddd


0.03460824711962095
0.032214906217410584
0.04975161794585531
0.027893995488846378
0.038656153056303565
0.07948802344218846
0.017545291875347546
0.040154873897707916
0.02759911428536519
0.044743943016541575


In [ ]:
# Compute \psi both ways
# Compute as *\phi
from geometry.numerical_star_R7 import Hodge_Dual

##train 'good' model for metric
##computed metric & 3-form on test data
psi = np.array([Hodge_Dual(predicted_g2forms[pt_idx], predicted_g2metrics[pt_idx]) for pt_idx in range(testsize)])

# Compute as 0.5 * \omega ^ \omega + Im(\Omega) ^ d\theta



In [ ]:
########################################################################################

In [ ]:
###testinggg
from geometry.geometry import wedge_product, wedge_form2_with_form1
fake_batchsize = 10
'''#tf
fake_2_forms = tf.random.normal((fake_batchsize, 6, 6))
fake_2_forms = fake_2_forms - tf.transpose(fake_2_forms, perm=[0, 2, 1])
fake_1_forms = tf.random.normal((fake_batchsize, 7))
'''
#np
fake_2_forms = np.random.randn(fake_batchsize, 6, 6)
fake_2_forms_66 = fake_2_forms - np.transpose(fake_2_forms, axes=[0, 2, 1])
fake_2_forms_77 = np.pad(fake_2_forms_66, ((0,0), (0,1), (0,1)), mode='constant')
fake_1_forms = np.random.normal(size=(fake_batchsize, 7))
print(f'Data shapes: {fake_2_forms.shape}, {fake_1_forms.shape}')

# old functionality:
output_old = wedge_form2_with_form1(fake_2_forms_66, fake_1_forms)
output_new = np.array([wedge_product(fake_2_forms_77[idx], fake_1_forms[idx]) for idx in range(fake_1_forms.shape[0])])

# scale output_old to match output_new
output_old *= 3 ###why is there this factor of 3 difference?

print(f'Output shapes: {output_old.shape}, {output_new.shape}')
print(f'Matching?? --> {np.allclose(output_old, output_new)}')